In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [6]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

ohe_geo = OneHotEncoder()
geo_encoded = ohe_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=ohe_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

# Split data into features and target variable
X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('ohe_geo.pkl', 'wb') as f:
    pickle.dump(ohe_geo, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)  

In [14]:
# Define a function to create the Keras model
def create_model(neurons=32, layers=1, dropout_rate=0.0):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
        # if dropout_rate > 0:
        #     model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [18]:
# Create a KerasClassifier
model = KerasClassifier(layers=1, neurons=32, build_fn=create_model, verbose=1)

In [19]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [20]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters and score
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits



c:\Users\james\development\ann-classification-churn\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)



Epoch 1/100


250/250 [==============================] - 1s 1ms/step - loss: 0.4944 - accuracy: 0.7875
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4296 - accuracy: 0.8125
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4142 - accuracy: 0.8219
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3997 - accuracy: 0.8338
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3850 - accuracy: 0.8399
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3727 - accuracy: 0.8480
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3630 - accuracy: 0.8511
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3562 - accuracy: 0.8553
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3520 - accuracy: 0.8565
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3488 - acc